In [4]:
import tensorflow as tf

In [6]:
from tensorflow.examples.tutorials.mnist import input_data

In [7]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
#INIT W

def init_weights(shape):
    init_rand_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_rand_dist)

In [9]:
#INIT BIAS

def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [10]:
#CONV2

def conv2d(x,W):
    # x --> [batch, H, W, Channels]
    # W --> [filter H, filter W, Channels IN, Channels OUT]
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [11]:
#POOLING

def max_pool_2by2(x):
    #x --> [batch, h, w, c]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [12]:
#CONVOLUTIONAL LAYER

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W) + b)

In [13]:
#NOMAL LAYER (FULLY CONNECTED)
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

In [14]:
#PLACEHOLDERS
x = tf.placeholder(tf.float32,shape=[None,784])
y = tf.placeholder(tf.float32,shape=[None,10])

In [15]:
#IMAGE
x_image = tf.reshape(x,[-1,28,28,1])

In [16]:
convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [17]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [18]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_1 = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [19]:
#DROPOUT
hold_prob = tf.placeholder(tf.float32)
full_1_dropout = tf.nn.dropout(full_layer_1,keep_prob=hold_prob)

In [20]:
y_pred = normal_full_layer(full_1_dropout,10)

In [21]:
#LOSS FUNCTION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [22]:
#OPTIMIZER
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [23]:
init = tf.global_variables_initializer()

In [ ]:
steps = 3000

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y:batch_y, hold_prob:0.5})
        
        if i % 100 == 0:
            
            print("ON STEP: {}".format(i))
            print("ACCURACY: ")
            matches = tf.equal(tf.arg_max(y_pred,1),tf.arg_max(y,1))
            
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            
            print(sess.run(acc,feed_dict={x:mnist.test.images,y: mnist.test.labels,hold_prob:1.0}))
            print("\n")
        

ON STEP: 0
ACCURACY: 
Instructions for updating:
Use `argmax` instead
